In [1]:
%matplotlib inline

import os
import sys
import json
import logging
import warnings
warnings.filterwarnings('ignore')

# 更改为自己的路径,或者直接在环境变量中添加
sys.path.append('/Users/pintec/Documents/repos/genie')

import pandas as pd
import numpy as np
from jinja2 import Template

#python3种reload函数不能直接使用
from imp import reload

from utils3.data_io_utils import *
import utils3.misc_utils as mu
import utils3.summary_statistics as ss
import utils3.metrics as mt
import utils3.feature_selection as fs
import utils3.plotting as pl
import utils3.modeling as ml
import utils3.filing as fl
from functools import reduce


/Users/pintec/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 构建取数逻辑（一次性整理成监控所需模样，减少后续工作量）
每个人取值逻辑不同，可根据自身情况调整，该代码可在监控配置中复用，此外表头必须含有apply_id,var_code,create_at字段,
    大家严格按照部署监控时刻的要求撰写PSI监控时刻的SQL取数则不会出错，如果唯一标识不是apply_id，务必将唯一标识字段名修改为apply_id

In [2]:
PSI_SQL = '''select userid as apply_id,curr_hismaxdpd,curoverdueday,currentosprincipal, '' as groupby_name
, '' as groupby_value,created_at,var_code,var_val from (
with application as
(
 select userid from hive_pro.hdp_data_rpt.hdp_rpt_ploan_application 
    where productid in ('DM-QBXJ') and ml_tasktype in ('CREDIT','ACTIVATE') 
    and bl_applicationdate < cast(current_date - interval '3' month as varchar)
    and approvedflag=1
    group by 1
)
, 
details as (
select userid, var_code, var_val, created_at from (
with a as (
select apply_id, max(case var_code when 'passportUserId' then var_val else null end) userid
from hive_pro.hdp_data_ods.hdp_ods_m_variable_dict where dt between '{{start_date}}' and '{{end_date}}' and product_name = 'walletAppBscore' group by apply_id
),
b as (
select apply_id, var_code, var_val, created_at,
row_number() over(partition by apply_id, var_code order by created_at desc) as rn
from hive_pro.hdp_data_ods.hdp_ods_m_variable_dict 
where dt between '{{start_date}}' and '{{end_date}}' and product_name = 'walletAppBscore'
and var_code in {{var_list}})
select a.userid,b.* from a left join b on a.apply_id = b.apply_id order by 1,2,3) where rn=1
),
curr_performance as (
select userid,max(curoverdueday) curoverdueday, max(hismaxdpd) curr_hismaxdpd, min(drawdownstamp) as drawdownstamp, sum(currentosprincipal) as currentosprincipal
from hive_pro.hdp_data_rpt.hdp_rpt_loan_performance
where dt= '{{per_date}}' and channeldetailcode in ('W2','DM-QBXJ')
group by userid
 
)
select application.userid, details.var_code, details.var_val, details.created_at
, curr_performance.curr_hismaxdpd,curr_performance.curoverdueday,curr_performance.currentosprincipal
from application inner join details
on application.userid = details.userid
inner join curr_performance 
on application.userid = curr_performance.userid) where curr_hismaxdpd<30 and curr_hismaxdpd>0 and curoverdueday=0 and currentosprincipal>0'''

# 取出部署时相关文件

In [3]:
# 部署时刻的评分卡,每个人路径不同,根据自己路径修改
score_card = pd.read_excel('/Users/pintec/Desktop/XJD_B_V3/03_部署文档/appbv3/自营个贷B卡_v3_2018.11.29.xlsx'\
                           ,sheetname='2_评分卡')

In [4]:
# 模型部署时刻decile,每个人路径不同,根据自己路径修改
model_decile = pd.read_excel('/Users/pintec/Desktop/XJD_B_V3/03_部署文档/appbv3/自营个贷B卡_v3_2018.11.29.xlsx'\
                           ,sheetname='3_decile')

In [5]:
# 部署时刻的rebin_spec文件,.json文件
rebin_spec_bin_adjusted = load_data_from_json('/Users/pintec/Desktop/XJD_B_V3/03_部署文档/appbv3'\
                                              ,'b_v3_selected_rebin_spec.json')

# 取数

In [6]:
# 将中间层指标以及打分以后的指标一并取出
var_list = set(list(score_card['中间层指标名称']) + list(score_card['输出打分指标名称']))
var_list = tuple(i for i in var_list if str(i)!='nan')

In [7]:
# 生成最后的sql,每个人情况不同,根据自身情况而定,但其中一定要有apply_id,var_code,create_at字段
final_sql = Template(PSI_SQL).render(start_date = '2018-12-01',end_date = '2018-12-08'\
                                     ,var_list = var_list,per_date = '2018-12-05')

In [8]:
data_B_v3 = presto_read_sql_df(final_sql)


# 生成上线报告相关数据

In [9]:
without_filling_miss_data,var_score_wrong,score_wrong,var_psi,score_psi = ss.verify_var_and_score_validity(data_B_v3,score_card,rebin_spec_bin_adjusted,model_decile)

In [10]:
FINAL_REPORT_PATH = '/Users/pintec/Desktop'

In [11]:
# python3.6的dict是ordered，按照顺序这样定义，之后生成excel的时候会按照顺序穿件sheet
data_dic = {}
# 如果sheet对应的内容是dict，则dict的key会出现在sheet第一列。value会从第二列开始插入
data_dic['01_变量未填充缺失值'] = without_filling_miss_data
data_dic['02_变量打分错误'] = var_score_wrong
data_dic['03_模型打分错误'] = score_wrong
data_dic['04_变量PSI'] = var_psi
data_dic['05_模型PSI'] = score_psi

In [12]:
fl.ModelSummary2Excel(FINAL_REPORT_PATH, None, 
                   '上线验证报告,xlsx', data_dic).run()